In [6]:
import numpy as np
import pandas as pd
import os

computer_path= os.getcwd()

path_input_data = computer_path + os.path.sep+ r"data"
path_EPCs = computer_path + os.path.sep+ r"data" + os.path.sep + r"EPCs"
saving_path = computer_path

### Functions used to extract the number of dwelling type and heating system from each EPC

In [7]:
def heatingSystem(df):

    df["Heat system"] = None
    df["MAINHEAT_DESCRIPTION"].fillna("No value", inplace=True)

    gas_list = ["mains gas (not community)",
                #'mains gas - this is for backwards compatibility only and should not be used'
                ]
    
    df.loc[df["MAIN_FUEL"].isin(gas_list), "Heat system"] = "Gas boiler"

    electricity_list = ["electricity (not community)", 'electricity - this is for backwards compatibility only and should not be used',
                       'Electricity: electricity, unspecified tariff']
    
    df.loc[df["MAIN_FUEL"].isin(electricity_list), "Heat system"]='Resistance heating'
    df.loc[df["MAINHEAT_DESCRIPTION"].str.contains("heat pump"),"Heat system"]="Heat pump"
    
    oil_list = ['appliances able to use mineral oil or liquid biofuel','oil (not community)', 'oil - this is for backwards compatibility only and should not be used']
    
    df.loc[df["MAIN_FUEL"].isin(oil_list), "Heat system"] = "Oil boiler"

    solid_fuels = ['anthracite', 'house coal (not community)','wood logs', 'smokeless coal', 'house coal - this is for backwards compatibility only and should not be used',
                  'bulk wood pellets', 'wood chips']

    df.loc[df["MAIN_FUEL"].isin(solid_fuels), "Heat system"]="Solid fuel boiler"
    
    df["Current heating costs [£]"] = df["HEATING_COST_CURRENT"]
    df["Potential heating costs [£]"] = df["HEATING_COST_POTENTIAL"]
    return df


def dwellingType(df):
    
    df["Dwelling type"]=None
    df.loc[df["PROPERTY_TYPE"]=="Flat", "Dwelling type"]="Flat"
    df.loc[df["PROPERTY_TYPE"]=="House", "Dwelling type"]=df.loc[df["PROPERTY_TYPE"]=="House", "BUILT_FORM"]
    list_dwelling_type = list(df["Dwelling type"].unique())
    list_dwelling_type = list(filter(None.__ne__, list_dwelling_type)) # Remove None from the list
    # temp_df.replace({dwel:"Terrace" for dwel in list_dwelling_type if "Terrace" in dwel}, regex=True, inplace=True)
   
    to_replace = {dwel:"Terraced" for dwel in list_dwelling_type if "Terrace" in dwel}
    df.replace(to_replace, inplace=True)
    df.replace("Semi-Detached", "Semi-detached", inplace=True)
    list_dwelling_type = list(df["Dwelling type"].unique())
    print(list_dwelling_type)
    
    to_drop = df.loc[(df["Dwelling type"]=='NO DATA!'),:].index
    df.drop(to_drop, inplace=True)
    
    df["Key"] = df["Dwelling type"]+" "+df["Heat system"]
    df["Key_EPC"] = df["CURRENT_ENERGY_RATING"]+" to "+df["POTENTIAL_ENERGY_RATING"]
    df.dropna(subset=['Key'], inplace=True)
    print("Shape:", df.shape)
    
    
    return df

def getEPC(path, file):
    df = pd.read_csv(path+os.path.sep+file)

    df.replace('Bungalow', 'House', regex=True, inplace=True)
    df.replace('Maisonette', 'Flat', regex=True, inplace=True)
    df["Heat consumption (kWh)"]=0
    df["Space constrained ratio"]=0
    df["Heat_rating_changes [%]"]=0
    df["Heat_efficiency_changes [%]"]=0
    print(df.shape)
    return df

### Link the EPC certificate file with the recommendation file to associate each measure to a dwelling category

In [8]:
# lookup table used to improve consistency of the naming of the energy efficiency measures
lookup_measures_df = pd.read_excel(path_input_data +os.path.sep+"Lookup_table_missing_description_measures_WITH_FORMULAS.xlsx")
lookup_measures_dict=dict(zip(lookup_measures_df["IMPROVEMENT_DESCR_TEXT"],lookup_measures_df["IMPROVEMENT_ID_TEXT"]))

In [9]:
# import costs of energy efficiency measures
costs_measures_df = pd.read_csv(path_input_data + os.path.sep+"costs_domestic_efficiency_measures_by_dwelling.csv")
costs_measures_df.columns = costs_measures_df.columns.str.strip()
costs_measures_df.head()

,Dwelling type,Measures,key,Costs,Comment,Source
0,Detached,50 mm internal or external wall insulation,Detached 50 mm internal or external wall insul...,10325.0,EWI+IWI avg costs,DECC Domestic Retrofit Cost Assumptions Study
1,Flat,50 mm internal or external wall insulation,Flat 50 mm internal or external wall insulation,4575.0,EWI+IWI avg costs,DECC Domestic Retrofit Cost Assumptions Study
2,Semi-detached,50 mm internal or external wall insulation,Semi-detached 50 mm internal or external wall ...,8650.0,EWI+IWI avg costs,DECC Domestic Retrofit Cost Assumptions Study
3,Terraced,50 mm internal or external wall insulation,Terraced 50 mm internal or external wall insul...,5825.0,EWI+IWI avg costs,DECC Domestic Retrofit Cost Assumptions Study
4,Detached,Add additional 80 mm jacket to hot water cylinder,Detached Add additional 80 mm jacket to hot wa...,90.0,hot water cylinder insulation cost,DECC Domestic Retrofit Cost Assumptions Study


In [16]:
file_measures = 'recommendations.csv'
file_epc = 'certificates.csv'

list_measures = []
list_missing_measures = []

for fn in os.listdir(path_EPCs):
#     for LA in ["Swansea"]:
#         if LA in fn:
            print(fn)
            df = pd.read_csv(path_EPCs+os.path.sep+fn+os.path.sep+file_measures, encoding='utf-8-sig')
            
            epc_df = pd.read_csv(path_EPCs+os.path.sep+fn+os.path.sep+file_epc, encoding='utf-8-sig')
            
            LA_name = fn.split('-')[2:]
            LA_name = ''.join(LA_name)
            print(LA_name, fn)
            temp_path = path_EPCs+os.path.sep+fn+os.path.sep
            epc_df = getEPC(temp_path, file_epc)
            epc_df = heatingSystem(epc_df)
            epc_df = dwellingType(epc_df)
            
            df.loc[(~df["IMPROVEMENT_DESCR_TEXT"].isna()) & (df["IMPROVEMENT_ID_TEXT"].isna()), "IMPROVEMENT_ID_TEXT"] = df["IMPROVEMENT_DESCR_TEXT"]
            df.replace({"IMPROVEMENT_ID_TEXT":lookup_measures_dict}, inplace=True)
                      
            print("length missing data:", len(df.loc[(~df["IMPROVEMENT_DESCR_TEXT"].isna()) & (df["IMPROVEMENT_ID_TEXT"].isna()), "IMPROVEMENT_DESCR_TEXT"].drop_duplicates()))

            df = pd.merge(df[["LMK_KEY", "IMPROVEMENT_ID_TEXT"]], epc_df[["LMK_KEY", "Key", "Dwelling type"]], right_on="LMK_KEY", left_on="LMK_KEY", how='left')
            df.dropna(inplace=True)
            
            measures_df = df["IMPROVEMENT_ID_TEXT"].value_counts().reset_index()
            
            to_merge_df = epc_df["Key"].value_counts().reset_index()
            to_merge_df.columns = ["Key", "Total"]

            measures_df = df.groupby(["Key", "IMPROVEMENT_ID_TEXT", "Dwelling type"]).count()["LMK_KEY"].reset_index()
            measures_df.columns = ["Key", "Measures", "Dwelling type", "Number"]
            measures_df = pd.merge(measures_df,to_merge_df, left_on="Key", right_on="Key", how='left') 

            measures_df["Key_measures"] = measures_df["Dwelling type"] + " " +measures_df["Measures"].astype(str)
            measures_df = pd.merge(measures_df, costs_measures_df[["key", "Costs"]], left_on="Key_measures", right_on="key", how="left")
#             measures_df.dropna(inplace=True)

            list_missing_measures= list_missing_measures +list(measures_df.loc[measures_df["Costs"].isna(), "Measures"].unique())
    
            measures_df["Total costs [£]"] = measures_df["Number"]*measures_df["Costs"]
            number_dwellings = measures_df[["Key", "Total"]].drop_duplicates().set_index("Key").sort_index()

            final_df = measures_df.groupby("Key").sum()[["Total costs [£]", "Number"]].sort_index()
            final_df.columns = ["Total costs [£]", "Number of measures"]
            
            final_df["Number of EPCs"] = number_dwellings["Total"].values
            final_df["Average costs [£]"] = final_df["Total costs [£]"]/final_df["Number of EPCs"]
            
            final_df.to_csv(saving_path+os.path.sep+LA_name+"_EPC_costs.csv", encoding='utf-8-sig')

domestic-W06000015-Cardiff
Cardiff domestic-W06000015-Cardiff
(101621, 89)
['Terraced', 'Semi-detached', 'Flat', 'Detached', 'NO DATA!']
Shape: (65648, 95)
length missing data: 0


In [17]:
final_df

,Total costs [£],Number of measures,Number of EPCs,Average costs [£]
Key,,,,
Detached Gas boiler,23960050.0,28378,5384,4450.232169
Detached Heat pump,19882.5,75,27,736.388889
Detached Oil boiler,555890.0,505,82,6779.146341
Detached Resistance heating,689477.0,831,306,2253.192810
Detached Solid fuel boiler,197460.0,166,36,5485.000000
Flat Gas boiler,13244895.0,25985,9925,1334.498237
Flat Heat pump,4000.0,31,39,102.564103
Flat Oil boiler,8705.0,37,18,483.611111
Flat Resistance heating,12315359.0,21947,12351,997.114323
